# Introduction
Research suggests that speech production can be modeled as a nonlinear dynamical system, wherein small perturbations in the interaction of its parts give rise to chaotic yet deterministic behavior. Parkinson's-related impairments (e.g. tremors, etc.) to the vocal organs, muscles and nerves can affect dynamics of the entire system, suggesting that nonlinear measures  may benefit the prediction of disease stage from voice recordings. 

## Features
### Traditional measures:
  * **shimmer** - extent of variation in amplitude from vocal cycle to vocal cycle
  * **noise-to-harmonics ratio (NHR)** -  amplitude of noise relative to tonal components of speech signal
  * **jitter** - measures pitch variation, such as vibrato and microtremor; calculated as differences in absolute frequencies of each cycle, averaged over a number of cycles
      - *Note*: Natural pitch variation exists in healthy individuals, but may be perturbed in those with vocal impairments secondary to Parkinson's. 
  
### Complex dynamical systems-based measures:

  * **correlation dimension** - used to recreate all possible states (phase space) of the system that generates speech  
  * **recurrence period density entropy (RPDE)** - this entropy measures the periodicity of the system
      - When the signal deviates from its trajectory of recurring to the same point in the phase space, this may indicate a voice disorder. Many voice disorders impair the patient's ability to sustain vocal fold vibration, which can be measured as in terms of aperiodicity.  
    
  * **detrended fluctuation analysis (DFA)** - extent of stochastic self-similarity of noise in the speech signal
      - Air blowing over vocal folds is a major cause of noise in speech, the pattern of which may be disrupted in some voice disorders. This noise can be characterized by a scaling exponent, which is higher in those with vocal disorders.  
      
  * **pitch period entropy (PPE)** - this entropy provides another measure of pitch variation (compare to **jitter**)
      - Because pitch is produced and perceived on a logarithmic scale, PPE is calculated first by converting a pitch sequence to the logarithmic semitone scale. A filter then removes natural pitch variations (such as those due to gender and individual differences), and a probability distribution of voice variations is constructed. Finally, entropy is calculated, characterizing the extent of variation beyond natural fluctuations in pitch. Increased PPE may suggest speech variations beyond those seen in healthy speech production.
      
### Target feature:

We will predict total scores on the [Unified Parkinson's disease rating scale (UPDRS)](https://neurosurgery.mgh.harvard.edu/functional/pdstages.htm), the scale most commonly used to study the long-term course of the disease. 

# Setting Up

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import warnings 
import datetime
warnings.simplefilter(action='ignore', category=FutureWarning)
sns.set()
pd.options.display.max_columns=None

In [ ]:
'''column 1: Subject id 

colum 2-27: features 
features 1-5: Jitter (local),Jitter (local, absolute),Jitter (rap),Jitter (ppq5),Jitter (ddp), 
features 6-11: Shimmer (local),Shimmer (local, dB),Shimmer (apq3),Shimmer (apq5), Shimmer (apq11),Shimmer (dda), 
features 12-14: AC,NTH,HTN, 
features 15-19: Median pitch,Mean pitch,Standard deviation,Minimum pitch,Maximum pitch, 
features 20-23: Number of pulses,Number of periods,Mean period,Standard deviation of period, features 24-26: Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks 

column 28: UPDRS 
column 29: class information 
'''
data_path = '../Data/classification/'
col_names = "subject_id,Jitter (local),Jitter (local. absolute),Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),Shimmer (local. dB),Shimmer (apq3),Shimmer (apq5), Shimmer (apq11),Shimmer (dda),AC,NTH,HTN,Median pitch,Mean pitch,Standard deviation,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS,class information".split(',')
df = pd.read_csv(data_path+"train_data.txt",names=col_names)
df

# Exploratory Data Analysis

## Descriptive statistics and data cleaning

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
corr = df.corr()
sns.heatmap(corr)
plt.title("Heat map of the correlation of the features")
plt.plot()

The distribution of the people with and without disease is exactly half:-

In [ ]:
df["class information"].value_counts()

Since we were unable to extract certain vocal features, we had to drop several columns and make do with what can extract.

Apart from vocal features, we drop subject id, because we cannot use this

In [ ]:
columns_to_drop = "subject_id,AC,NTH,HTN,Median pitch,Mean pitch,Standard deviation,Minimum pitch,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks".split(',')
#columns_to_drop = ["subject_id"]
def to_target(val):
    '''
    Converting class 0 and 1 to False and true respectively
    '''
    if val==0:
        return False
    return True
df.UPDRS = df.UPDRS.astype('float')
df_new = df.drop(columns=columns_to_drop)

df_new["class_information"] = df_new["class information"].apply(to_target)
df_new.drop(columns=['class information'],inplace=True)
df_new

In [ ]:
del(df)

In [ ]:
sns.distplot(df_new["UPDRS"])
plt.title("Distribution of the UPDRS scores among the subjects")
plt.show()

We can notice that most subjects have a UPDRS score less than 5 while the rest are above that. This is because half of the data consists of subjects without parkinsons's disease(PD). We can consider a subject to not have PD if his UPDRS score is less than 5.

In [ ]:
for z in df_new.drop(columns=['class_information','UPDRS']).columns:
    ax = sns.violinplot(x="class_information", y=z,  bw=.2, palette={True: "#f9e9e8", False: "#CD5C5C"}, inner="quartile", data=df_new)
    ax.set_xticklabels(['Not Diseased', 'Diseased'])
    plt.show()

In [ ]:
g = sns.pairplot(df_new,vars=df_new.drop(columns=['class_information',"UPDRS"]).columns, hue='class_information',palette={True: "green", False: "blue"})
#g = sns.pairplot(df_new, vars=df_new.drop(columns='sex').columns, hue="sex")
g.map_diag(plt.hist)
g.map_lower(sns.regplot,fit_reg=False)
g.map_upper(sns.kdeplot, cmap="Blues_d")
#g.map_offdiag(sns.kdeplot, cmap="Blues_d", n_levels=6);
plt.show()

# Statistical analysis

First lets import the modules required for statistical tests

In [ ]:
from scipy.stats import chi2_contingency

Since the target variable is a categorical variable, we can use the chi square test for determining the import features. However, for this we first need to convert the continuous variables into categorical variables. In order to do this, we shall put the values into bins. In order to get the best number of bins, we use the Freedman-Diaconis formula

In [ ]:
_,bins_edges = np.histogram(df_new.iloc[:,0],bins='fd')
bins_edges,len(bins_edges)

As we see above, we get 39 different bins with minimum loss of the data

Lets create a new dataframe for the analysis

In [ ]:
stats_df = df_new.copy()
for x in stats_df.drop(columns = ["class_information","UPDRS"]).columns:
    _,bins_edges = np.histogram(stats_df.loc[:,x],bins='fd')
    bins_edges,len(bins_edges)
    stats_df.loc[:,x] = pd.cut(stats_df.loc[:,x],bins=bins_edges).astype('str')
stats_df

#### Now that we have categorical variables, we can proceed with the chi square test of independence. The hypothesis is as follows:-
* Null Hypothesis : There is no relationship between the two variables
* Alternate hypothesis : There is a relationship between the two variables

After performing the chi square test, we look at the p value and decide if the value is significant or not. We will go ahead with the standard confidence limit alpha = 0.05. So if p value is less than 0.05, then we can reject the null hypothesis.<br>
Since there are multiple columns, lets build a simple wrapper function to perform the test on all the required columns at once.

In [ ]:
def chi_square_wrapper(df,target,columns=None,verbose=False,alpha=0.05):
    '''Function performs the Chi square test of independence of all the columns with the target variable.
    This test is generally used for comparing two categorical variables.
    
    Parameters
    ----------
    df -> The pandas dataframe
    target -> The target variable you want to test for
    columns -> the specific list of columns you want test. All if not specified.
    alpha
    '''
    if columns == None:
        columns=[x for x in df.columns if df[x].dtype=='object']
    for column in columns:
        assert df[column].dtype == 'object',"This function is intended for categorical indipendent variables"
    useful_columns = []
    for column in columns:
            if column == target:
                continue
            print("For "+column+" :-")
            cros = pd.crosstab(stats_df.loc[:,column],stats_df.loc[:,target])
            res = chi2_contingency(cros)
            if verbose:
                print(res)
            if res[1] < alpha :
                print("Reject null hypothesis\n--------------------")
                useful_columns.append(column)
            else:
                print("Accept null hypothesis\n--------------------")
    print("According to the test, the useful columns are :-\n"+", ".join(useful_columns))

Lets perform the test and see which columns are useful

In [ ]:
chi_square_wrapper(stats_df,'class_information')

In [ ]:
del(stats_df)

The Chi square test shows these columns are significant. Now lets proceed to build the model.

# Model Building

First import the required modules

In [ ]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier,AdaBoostClassifier,GradientBoostingClassifier,VotingClassifier,RandomForestClassifier,BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer, Normalizer
from sklearn.decomposition import PCA
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

There are certain preprocessing steps that help some model perform better. Lets create a wrapper for the pipeline creation for this.

In [ ]:
def create_pipe_model(model,scale=preprocessing.Normalizer()):
    '''
    Returns a pipeline with the 4 steps:-
    1)Scaling the data
    2)Feature selection or dimentionality reducion
    3)Regression
    '''
    return Pipeline([
                  ('scale',scale),
                  ('feature_selection', SelectFromModel(model)),
                  ('Classification', model)
                    ])
def run_model(pipe,param_dict,verbose=True,n_jobs=-1,n_iter=100):
    '''
    pipe -> The pipeline or the model for which we want the randomised search to run
    param_dict -> The dictionary of parameters.
    Note : for parameter of a particular stage of a pipleline, use stage_name__param_name as the key. For example Classification__kernel for the classification stage kernel parameter
    '''
    clf = RandomizedSearchCV(pipe,param_dict,verbose=verbose,n_jobs=n_jobs,error_score=0.0,n_iter=n_iter).fit(X_train,y_train)
    print("Training score : ",clf.score(X_train,y_train))
    print("Testing score : ",clf.score(X_test,y_test))
    print("Best params : ",clf.best_params_)
    return clf

Now that the pipline wrapper is ready, lets segregate the features and targets and create the training and testing sets

In [ ]:
targets=df_new['class_information']
features = df_new.drop(columns=['class_information','UPDRS'])
features.head()

In [ ]:
targets.head()

The stratify option allows to get equal split of classes in training and testing sets

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(features,targets,random_state=2,test_size=0.2,stratify=targets)

In [ ]:
clfs = {}  #stores all the models
scalers_to_test = [StandardScaler(), RobustScaler(), QuantileTransformer(),Normalizer(),None]

## Now that the features are ready, let's start building the models

### Models considered are:-
<ol>
    <li> Naive Bayes </li>
    <li> Logistic Regression </li>
    <li> Decision Tree</li>
    <li> Nearest Neighbours</li>
    <li> SVM </li> 
    <li> Neural Networks </li>
</ol>

### And some ensembles such as :-
<ol>
    <li> Random Forest</li>
    <li> Extra trees</li>
    <li> Ada boost</li>
    <li> Gradient boost</li>
    <li> Bagging estimators</li>
    <li> XG boost </li>        
</ol>

For each of the models, we will use the randomized search to find the best possible hyperparameters. It is computationally expensive but we will not miss out on any model beacuse of poor hyperparameter tuning. Let's also calculate the total time taken to train the models using this method. Time taken for individual models is also displayed. 

In [ ]:
from time import time
start_time = time()

### Naive Bayes

In [ ]:
model = GaussianNB()
feature_selection = [SelectFromModel(model),None]+[PCA(x) for x in range(1,11,3)]
params = {
    'scale' : scalers_to_test,
    'feature_selection' : feature_selection
}
pipe = create_pipe_model(model)
clfs["Naive_Bayes"] = run_model(pipe,params,n_iter=30)

### Logistic Regression

In [ ]:
model = LogisticRegression()
feature_selection = [SelectFromModel(model),None]+[PCA(x) for x in range(1,11,3)]
params = {
    'scale' : scalers_to_test,
    'feature_selection' : feature_selection,
    'Classification__penalty' : ['l1','l2'],
    'Classification__C' : [x for x in range(1,5)],
    'Classification__solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}
pipe = create_pipe_model(model)
clfs["Logistic_regression"] = run_model(pipe,params)

### Decision trees

In [ ]:
model = DecisionTreeClassifier()
feature_selection = [SelectFromModel(model),None]+[PCA(x) for x in range(1,11,3)]
pipe = create_pipe_model(model)
params = {
    'scale' : scalers_to_test,
    'feature_selection' : feature_selection,
    'Classification__criterion' : ['gini','entropy'],
    'Classification__splitter' : ["best",'random'],
    'Classification__max_depth' : [x for x in range(8,20,3)]+[None],
    'Classification__max_features' : ['auto','sqrt','log2',None],
    'Classification__min_samples_split' : [x for x in range(2,10,2)],
}
clfs["Decision_tree"] = run_model(pipe,params,verbose=True)

### Nearest Neighbours

In [ ]:
model = KNeighborsClassifier()
feature_selection = [SelectFromModel(model),None]+[PCA(x) for x in range(1,11,3)]
pipe = create_pipe_model(model)
params = {
    'scale' : scalers_to_test,
    'feature_selection' : feature_selection,
    'Classification__n_neighbors' : [x for x in range(1,10,2)],
    'Classification__algorithm' : ['auto', 'ball_tree', 'brute','kd_tree']
}
clfs["K_nearest"] = run_model(pipe,params,verbose=True)  #Setting n_jobs as 6 so that i can continue working on my system without any lag:P

### SVM

In [ ]:
model = SVC(probability=True)
feature_selection = [None]+[PCA(x) for x in range(1,11,3)]
pipe = create_pipe_model(model)
params = {
    'scale' : scalers_to_test,
    'feature_selection' : feature_selection,
    'Classification__C' : [x for x in range(1,10,2)],
    'Classification__kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
}
clfs["SVM"] = run_model(pipe,params,verbose=True,n_iter=10,n_jobs=2)

### Neural networks

In [ ]:
model = MLPClassifier()
feature_selection = [None]+[PCA(x) for x in range(1,11,3)]
pipe = create_pipe_model(model)
params = {
    'scale' : scalers_to_test,
    'feature_selection' : feature_selection,
    'Classification__hidden_layer_sizes' : [(100,200,250,400),(100,),(200,360,300)],
    'Classification__activation': ["logistic", "relu", "tanh",'identity'],
}
clfs["Neural_networks"] = run_model(pipe,params,verbose=True,n_iter=100)

### Random Forest

In [ ]:
model = RandomForestClassifier()
feature_selection = [SelectFromModel(model),None]+[PCA(x) for x in range(1,11,3)]
pipe = create_pipe_model(model)
params = {
    'scale' : scalers_to_test,
    'feature_selection' : feature_selection,
    'Classification__n_estimators' : [10,100,500,1000,1500,2000,3000]#[x for x in range(10,5000,100)],
    'Classification__criterion' : ['gini','entropy'],
    'Classification__max_depth' : [x for x in range(8,15,4)]+[None],
    'Classification__max_features' : ['auto','sqrt','log2',None],
    'Classification__bootstrap' : [True,False],
}
clfs["Random_forest"] = run_model(pipe,params,verbose=True)

### Extra trees 

In [ ]:
model = ExtraTreesClassifier()
feature_selection = [SelectFromModel(model),None]+[PCA(x) for x in range(1,11,3)]
pipe = create_pipe_model(model)
params = {
    'scale' : scalers_to_test,
    'feature_selection' : feature_selection,
    'Classification__n_estimators' : [10,100,500,1000,1500,2000,3000],
    'Classification__criterion' : ['gini','entropy'],
    'Classification__max_depth' : [x for x in range(8,15,4)]+[None],
    'Classification__max_features' : ['auto','sqrt','log2',None],
    'Classification__bootstrap' : [True,False],
}
clfs["Extra_trees"] = run_model(pipe,params,verbose=True)

### ADA Boost

In [ ]:
model = AdaBoostClassifier()
feature_selection = [SelectFromModel(model),None]+[PCA(x) for x in range(1,11,3)]
pipe = create_pipe_model(model)
params = {
    'scale' : scalers_to_test,
    'feature_selection' : feature_selection,
    'Classification__n_estimators' : [10,100,500,1000,1500,2000,3000],
    'Classification__base_estimator' : [GaussianNB(),LogisticRegression(),KNeighborsClassifier(),DecisionTreeClassifier(),SVC()],
}
clfs["Ada_boost"] = run_model(pipe,params,verbose=True)

### Gradient Boost

In [ ]:
model = GradientBoostingClassifier()
feature_selection = [SelectFromModel(model),None]+[PCA(x) for x in range(1,11,3)]
pipe = create_pipe_model(model)
params = {
    'scale' : scalers_to_test,
    'feature_selection' : feature_selection,
    'Classification__n_estimators' : [10,100,500,1000,1500,2000,3000],
    'Classification__loss' : ['deviance', 'exponential'],
    'Classification__max_depth' : [x for x in range(3,15,4)]
}
clfs["Gradient_boost"] = run_model(pipe,params,verbose=True)

### Bagging Estimator

In [ ]:
model = BaggingClassifier()
feature_selection = [SelectFromModel(model),None]+[PCA(x) for x in range(1,11,3)]
pipe = create_pipe_model(model)
params = {
    'scale' : scalers_to_test,
    'feature_selection' : feature_selection,
    'Classification__n_estimators' : [10,100,500,1000,1500,2000,3000],
    'Classification__base_estimator' : [GaussianNB(),LogisticRegression(),KNeighborsClassifier(),DecisionTreeClassifier(),SVC()],
}
clfs["Ada_boost"] = run_model(pipe,params,verbose=True)

### XG BOOST

In [ ]:
model = XGBClassifier()
feature_selection = [SelectFromModel(model),None]+[PCA(x) for x in range(1,11,3)]
pipe = create_pipe_model(model)
params = {
    'scale' : scalers_to_test,
    'feature_selection' : feature_selection,
    'Classification__n_estimators' : [10,100,500,1000,1500,2000,3000],
    'Classification__max_depth' : [x for x in range(3,15,4)],
    'Classification__booster' : ['gbtree', 'gblinear','dart']
}
clfs["XGboost"] = run_model(pipe,params,verbose=True)

### Voting Classifier

In [ ]:
model = VotingClassifier([('gb',GaussianNB()),('lr',LogisticRegression()),('knn',KNeighborsClassifier()),('dt',DecisionTreeClassifier()),('svm',SVC(probability=True))])
feature_selection = [SelectFromModel(model),None]+[PCA(x) for x in range(1,11,3)]
pipe = create_pipe_model(model)
params = {
    'scale' : scalers_to_test,
    'feature_selection' : feature_selection,
    'Classification__voting' : ['hard','soft'],
    'Classification__estimators':[[('gb',GaussianNB()),('lr',LogisticRegression()),('knn',KNeighborsClassifier()),('dt',DecisionTreeClassifier()),('svm',SVC(probability=True))]]
}
#clfs["Voting_classifier"] = run_model(pipe,params,verbose=True)
Voting_classifier = run_model(pipe,params,verbose=True,n_iter=60)

## Finally,
### Let's try the ensemble of all the classifier considered so far

In [ ]:
class ensemble(object):
    """Stripped-down version of VotingClassifier that uses prefit estimators"""
    def __init__(self, estimators, voting='hard', weights=None):
        self.estimators = [e[1] for e in estimators]
        self.named_estimators = dict(estimators)
        self.voting = voting
        self.weights = weights

    def fit(self, X, y, sample_weight=None):
        raise NotImplementedError
        
    def predict(self, X):
        """ Predict class labels for X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.

        Returns
        ----------
        maj : array-like, shape = [n_samples]
            Predicted class labels.
        """

        #check_is_fitted(self, 'estimators')
        if self.voting == 'soft':
            maj = np.argmax(self.predict_proba(X), axis=1)

        else:  # 'hard' voting
            predictions = self._predict(X)
            maj = np.apply_along_axis(lambda x:
                                      np.argmax(np.bincount(x,
                                                weights=self.weights)),
                                      axis=1,
                                      arr=predictions.astype('int'))
        return maj

    def _collect_probas(self, X):
        """Collect results from clf.predict calls. """
        return np.asarray([clf.predict_proba(X) for clf in self.estimators])

    def _predict_proba(self, X):
        """Predict class probabilities for X in 'soft' voting """
        if self.voting == 'hard':
            raise AttributeError("predict_proba is not available when"
                                 " voting=%r" % self.voting)
        avg = np.average(self._collect_probas(X), axis=0, weights=self.weights)
        return avg

    @property
    def predict_proba(self):
        """Compute probabilities of possible outcomes for samples in X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.

        Returns
        ----------
        avg : array-like, shape = [n_samples, n_classes]
            Weighted average probability for each class per sample.
        """
        return self._predict_proba

    def transform(self, X):
        """Return class labels or probabilities for X for each estimator.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.

        Returns
        -------
        If `voting='soft'`:
          array-like = [n_classifiers, n_samples, n_classes]
            Class probabilities calculated by each classifier.
        If `voting='hard'`:
          array-like = [n_samples, n_classifiers]
            Class labels predicted by each classifier.
        """
        if self.voting == 'soft':
            return self._collect_probas(X)
        else:
            return self._predict(X)

    def _predict(self, X):
        """Collect results from clf.predict calls. """
        return np.asarray([clf.predict(X) for clf in self.estimators]).T

In [ ]:
model = ensemble([(a,b) for a,b in clfs.items()])
yhat = model.predict(X_test)
y = y_test
print("Hard voting accuracy :-",accuracy_score(y,yhat))

### To do :-
* **Visualize results**
* **GUI**

In [ ]:
from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(list(y),list(model.predict(X_test)))
print(cm)

In [ ]:
def showFeatureImportance(model):
    #FEATURE IMPORTANCE
    # Get Feature Importance from the classifier
    feature_importance = model.feature_importances_

    # Normalize The Features
    feature_importance = 100.0 * (feature_importance / Feature_importance.max())
    sorted_idx = np.argsort(feature_importance)
    pos = np.arange(sorted_idx.shape[0]) + .5

    #plot relative feature importance
    plt.figure(figsize=(12, 12))
    plt.barh(pos, feature_importance[sorted_idx], align='center', color='#7A68A6')
    plt.yticks(pos, np.asanyarray(X_cols)[sorted_idx])
    plt.xlabel('Relative Importance')
    plt.title('Feature Importance')
    plt.show()